In [69]:
import nbimporter
import pandas as pd
import numpy as np
import warnings
import util_data_cleaner
import fetch_data_from_yfinance
warnings.filterwarnings('ignore')
from importlib import reload
reload(util_data_cleaner)
reload(fetch_data_from_yfinance)
from datetime import date
import yfinance as yf

In [31]:
ipo_listing_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\config\IPO-PastIssue-23-Aug-2024.csv')
ipo_listing_data = util_data_cleaner.clean_ipo_symbol_data(ipo_listing_data)
ipo_listing_data = ipo_listing_data.loc[ipo_listing_data['DATE OF LISTING']>='2020-01-01',]

In [122]:
b = []
for stock in ipo_list:
    start_date = pd.to_datetime(ipo_listing_data.loc[ipo_listing_data['Symbol']==stock,'DATE OF LISTING'].values[0])
    end_date = date.today()
    interval = '1d'
    a = fetch_data_from_yfinance.data_puller(stock, start_date, end_date, interval)
    b.append(a)
final_data = pd.concat(b)
final_data.reset_index(inplace=True)


1 Failed download:
['BALCO.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['SUNLITE.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['POSITRON.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['AESTHETIK.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['PPSL.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['DHARIWAL.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['UTSSAV.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['ASHALOG.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['SSEGL.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['BULKCORP.NS']: Exception('%ticker%: No timezone found, symbol may be deli

In [123]:
final_data.sort_values(by=['Stock Name', 'Date'], inplace=True)
final_data['Ipo_price'] = final_data.groupby('Stock Name')['Adj Close'].transform('first')
final_data['lower_bound'] = final_data['Ipo_price'] * 0.9
final_data['upper_bound'] = final_data['Ipo_price'] * 1.1
latest_close = final_data.groupby('Stock Name').tail(1)

In [125]:
latest_close['to_filter'] = 0

In [126]:
latest_close.loc[((latest_close['Adj Close'] >= latest_close['lower_bound']) & (latest_close['Adj Close'] <= latest_close['upper_bound'])),'to_filter'] = 1

In [130]:
latest_close.to_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\strategy results\ipo_bo_23_aug_2024.csv', index=False)